In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

C:\Users\Vikas\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import nltk
import math
from sklearn.model_selection import train_test_split
import numpy as np


In [4]:
filenames = ['whitman-leaves.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt','shakespeare-macbeth.txt']
with open('input.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [5]:
num_lines = sum(1 for line in open('input.txt'))
print(num_lines)
#split data into train test and validation


with open("input.txt", "r") as f:
    data = f.read().split('\n')

i=int(num_lines*0.7)

train_data = data[:i]
test_data = data[i:]

29166


In [6]:

thefile = open('train.txt', 'w')
thefile.write("\n".join(train_data))

806883

In [9]:
import string
 

def load_doc(filename):
        
        file = open(filename, 'r')
        # read all text
        text = file.read()
        # close the file
        file.close()
        return text


def clean_doc(doc):
        
        doc = doc.replace('--', ' ')
        
        tokens = doc.split()
        
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        
        tokens = [word for word in tokens if word.isalpha()]
        
        tokens = [word.lower() for word in tokens]
        return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
        data = '\n'.join(lines)
        file = open(filename, 'w')
        file.write(data)
        file.close()


in_filename = 'train.txt'#
doc = load_doc(in_filename)

 

tokens = clean_doc(doc)
#print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))
 

Total Tokens: 139632
Unique Tokens: 15297


In [79]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

Total Sequences: 139581


In [16]:

#load
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
#print(type(lines))
#print(lines[:5])
 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]


print("Total no of  Sequences",len(sequences))
#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(sequences)
#sequences = tokenizer.texts_to_sequences(sequences)
#vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size :',vocab_size)
#sequences=np.array(sequences)
#x,y=sequences[:,:-1],sequences[:,-1]
#seq_length = x.shape[1]

Total no of  Sequences 139581
Vocabulary size : 15298


In [17]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            764900    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 15298)             1545098   
Total params: 2,460,898
Trainable params: 2,460,898
Non-trainable params: 0
_________________________________________________________________
None


In [85]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=1000, epochs=30)
 
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Epoch 1/30
139581/139581 [==============================] - 1256s 9ms/step - loss: 6.7826 - acc: 0.0760
Epoch 2/30
139581/139581 [==============================] - 1141s 8ms/step - loss: 6.6737 - acc: 0.0818
Epoch 3/30
139581/139581 [==============================] - 1161s 8ms/step - loss: 6.6102 - acc: 0.0857
Epoch 4/30
139581/139581 [==============================] - 1153s 8ms/step - loss: 6.5644 - acc: 0.0870
Epoch 5/30
139581/139581 [==============================] - 830s 6ms/step - loss: 6.5275 - acc: 0.0878
Epoch 6/30
139581/139581 [==============================] - 641s 5ms/step - loss: 6.4906 - acc: 0.0886
Epoch 7/30
139581/139581 [==============================] - 640s 5ms/step - loss: 6.4496 - acc: 0.0892
Epoch 8/30
139581/139581 [==============================] - 635s 5ms/step - loss: 6.4051 - acc: 0.0930
Epoch 9/30
139581/139581 [==============================] - 645s 5ms/step - loss: 6.3615 - acc: 0.0974
Epoch 10/30
139581/139581 [==============================] - 650s 5ms

In [7]:
thefile1 = open('test.txt', 'w')
thefile1.write("\n".join(test_data))



279873

In [13]:
#texword sequences
in_filename = 'test.txt'#
doc = load_doc(in_filename)
#print(doc[:200])
 
# clean document
tokens_test = clean_doc(doc)
#print(tokens[:200])


In [14]:
length = 50 + 1
sequences_test = list()
for i in range(length, len(tokens_test)):
	# select sequence of tokens
	seq = tokens_test[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences_test.append(line)
print('Total Sequences: %d' % len(sequences_test))
 
# save sequences to file
out_filename = 'sequences_test.txt'
save_doc(sequences_test, out_filename)

Total Sequences: 50152


In [38]:
from keras.models import load_model
in_filename = 'sequences_test.txt'
doc = load_doc(in_filename)
lines = list(doc.split('\n'))


model = load_model('model.h5')

sequences_c = tokenizer.texts_to_sequences(lines)

sequences = []
for sequence in  sequences_c:
        if(len(sequence)==51):
            sequences.append(sequence)



sequences =  np.asarray(sequences)

X_test, y_test = sequences[:,:-1], sequences[:,-1]

y_test = to_categorical(y_test, num_classes=vocab_size)
 

loss = model.evaluate(X_test,y_test,batch_size=128, verbose=1)



220/220 [==============================] - 1s 6ms/step


In [50]:
import math


print("perplexity = {}" .format(math.exp(loss)))

perplexity = 221.40641620418717
